## <center>Introductory Econometrics - Data Task<center>
## <center>探討名字「菜市場程度」與考試成績的關聯<center>
### <center>汪晁安<center>

<br/><br/>
資料說明：\
    -`list.txt`：2004 年 8 月大學入學分發榜單 \
    -`depname.txt`：科系代碼表 \
\
使用Julia + Jupyter完成。

## 進行資料整理

In [2]:
using StringEncodings, DataFrames, CSV
rawlist = readlines(".\\list.txt", enc"UTF-16BE")      #須將檔案與ipynb置於同一資料夾才能讀取 且須先將格式轉換成UTF-16BE
rawschool = readlines(".\\depname.txt", enc"UTF-16BE")
Unilist = []
dptlist = []
list = []
dptCode = 1
N_uni = 0
N_stu = 0
P = 0
N = 0
flag = false
rawlist[1] = rawlist[1][4:8]
rawschool[1] = rawschool[1][4:end]

#科系代碼
push!(Unilist, [1, "國立臺灣大學", "001", 41, 1, 0]) #公立1 私立0 / 全名 / 累計錄取人數 / 學系代碼開頭 / 學系代碼結尾
N_stu += 41
for i in 2:length(rawschool)
    
    #逐行分割
    prev = filter(s->!all(isempty, s), split(rawschool[i - 1]," "))
    tmp = filter(s->!all(isempty, s), split(rawschool[i]," "))
    if length(tmp) == 3     
        push!(tmp, tmp[end][8:10], tmp[end][11:13])
        tmp[3] = tmp[3][1:7]
    elseif length(tmp) == 4
        push!(tmp, tmp[end][3:5])
    end
    
    #讀取
    if tmp[3][5:7] != Unilist[end][3]
        N_uni +=1
        Unilist[end][6] = parse(Int64, prev[3][1:4])
        push!(Unilist,[1 * (tmp[1][1:4] == "國立"), tmp[1], tmp[3][5:7],parse(Int64, tmp[5]), parse(Int64, tmp[3][1:4]), 0])
    else
        Unilist[end][4] += parse(Int64, tmp[5])
    end
    N_stu += parse(Int64, tmp[5])
    
    if i ==length(rawschool)
        Unilist[end][6] = parse(Int64, tmp[3][1:4])
        N_uni += 1
    end
end
sort!(Unilist, by = x -> x[4], rev=true)

#榜單
for i in 2:length(rawlist)
    flag = false
    tmp = split(rawlist[i], "    ")
    
    #取科系代碼
    if length(tmp) == 1 && tmp[1][1] == ' '
        dptCode = tmp[1][2:5]
        dptCode = parse(Int64, dptCode)
        flag = true
        for j in Unilist
            if dptCode <= j[6] && dptCode >= j[5]
                dptCode = j[1]
            end
        end
    end
    
    #讀取
    if !flag
        for k in 1:length(tmp) - 1
            if tmp[k] != ""
                tmp[k] = lstrip(tmp[k])
                push!(list,(tmp[k][12:end], dptCode))
            end
        end
    end
end

#印出整理過的資料 前十項
print(list[1:10],"\n") #名字 : 公立1 私立0
print(Unilist[1:10]) #公立1 私立0 / 全名 / 累計錄取人數 / 學系代碼開頭 / 學系代碼結尾

Any[("安沛", 1), ("芊曄", 1), ("姿瑩", 1), ("怡親", 1), ("仕達", 1), ("溫馨", 1), ("治平", 1), ("宗翰", 1), ("秀倩", 1), ("盈孜", 1)]
Any[Any[0, "中國文化大學", "017", 4296, 770, 1065], Any[0, "淡江大學", "014", 3587, 633, 701], Any[0, "輔仁大學", "020", 3394, 1131, 1225], Any[0, "銘傳大學", "046", 3197, 2156, 2211], Any[0, "逢甲大學", "015", 2938, 702, 741], Any[1, "國立臺灣大學", "001", 2866, 1, 76], Any[0, "東海大學", "009", 2809, 451, 534], Any[0, "真理大學", "079", 2426, 2522, 2557], Any[0, "東吳大學", "005", 2398, 254, 320], Any[0, "實踐大學", "050", 2379, 2252, 2315]]

## 觀察資料性質

In [3]:
print("大學總數: ",N_uni, "\n學生總數: " , N_stu, "\n\n錄取人數最多的前十個學校​: \n")
for i in 1:10
    print(Unilist[i][2],": ",Unilist[i][4], "人\n")
end

大學總數: 71
學生總數: 89035

錄取人數最多的前十個學校​: 
中國文化大學: 4296人
淡江大學: 3587人
輔仁大學: 3394人
銘傳大學: 3197人
逢甲大學: 2938人
國立臺灣大學: 2866人
東海大學: 2809人
真理大學: 2426人
東吳大學: 2398人
實踐大學: 2379人


In [4]:
for i in Unilist
    if i[1] == 1
        N += i[4]
    else
        P += i[4]
    end
end
print("公立學校共錄取", N, "人\n")
print("私立學校共錄取", P, "人\n")

公立學校共錄取28219人
私立學校共錄取60816人


In [5]:
NnameDict = Dict()
PnameDict = Dict()
for i in list
    if i[2] == 1
        if !haskey(NnameDict, i[1])
            NnameDict[i[1]] = 1
        else
            NnameDict[i[1]] += 1
        end
    else
        if !haskey(PnameDict, i[1])
            PnameDict[i[1]] = 1
        else
            PnameDict[i[1]] += 1
        end
    end
end

NsortDict = sort(collect(NnameDict), by = x -> x.second, rev = true)
PsortDict = sort(collect(PnameDict), by = x -> x.second, rev = true)
print("公立大學出現最多次的名字: ", NsortDict[1][1])
print("\n共出現",NsortDict[1][2],"次")
print("\n私立大學出現最多次的名字: ", PsortDict[1][1])
print("\n共出現",PsortDict[1][2],"次")

公立大學出現最多次的名字: 怡君
共出現73次
私立大學出現最多次的名字: 雅婷
共出現218次

### 令榜單中只出現一次的名字稱為「獨特名字」，計算公私立大學錄取學生的名字為「獨特名字」的比例，並檢驗是否有顯著差異。

In [7]:
#import Pkg; Pkg.add("HypothesisTests")
using Statistics, HypothesisTests
NpercDict = []
PpercDict = []
tmp = 0.0

#get percentages
for i in NsortDict
    push!(NpercDict, (i[1], round(i[2]/N; digits = 6)))
end
for i in PsortDict
    push!(PpercDict, (i[1], round(i[2]/P; digits = 6)))
end

for i in NpercDict
    if i[2] > 3.5e-5
        tmp += i[2]
    end
end
UniqRateN = round(1 - tmp; digits = 4)

tmp = 0.0
for i in PpercDict
    if i[2] > 1.6e-5
        tmp += i[2]
    end
end
UniqRateP = round(1 - tmp; digits = 4)

print("公立大學獨特名字的比例: \n")
print(UniqRateN)
print("\n私立大學獨特名字的比例: \n")
print(UniqRateP)
print("\n\n公立大學的比例較高\n\n")

#generate vectors for t-test
Ntest = []
Ptest = []
for i in list
    if i[2] == 1
        if NnameDict[i[1]] == 1
            push!(Ntest, 1)
        else
            push!(Ntest, 0)
        end
    elseif i[2] == 0
        if PnameDict[i[1]] == 1
            push!(Ptest, 1)
        else
            push!(Ptest, 0)
        end
    else
        print(i)
    end
end

Ntest = convert(Array{Int64,1}, Ntest)
Ptest = convert(Array{Int64,1}, Ptest)
print(UnequalVarianceTTest(Ntest, Ptest))
print("------------------------------------\n\noutcome with 95% confidence: reject h_0 -> 有顯著差異.")

n公立大學獨特名字的比例: 
0.5204
私立大學獨特名字的比例: 
0.3959

公立大學的比例較高

Two sample t-test (unequal variance)
------------------------------------
Population details:
    parameter of interest:   Mean difference
    value under h_0:         0
    point estimate:          0.124522
    95% confidence interval: (0.1175, 0.1315)

Test summary:
    outcome with 95% confidence: reject h_0
    two-sided p-value:           <1e-99

Details:
    number of observations:   [28219,60816]
    t-statistic:              34.8352396782212
    degrees of freedom:       53944.272826798435
    empirical standard error: 0.0035745872159658404
------------------------------------

outcome with 95% confidence: reject h_0 -> 有顯著差異.

### 定義常見名字的累積比例為其集中度，公私立學校何者的集中度較高？

In [10]:
print("(f):\n公立大學中出現次數最多的前十個名字: \n")
tmp = 0.0
print(N)
for i in NsortDict[1:10]
    tmp += round(i[2]/N; digits = 4)
    print(i[1], " /出現次數: ", i[2], " /累積比例: ", round(tmp; digits = 4), "\n")
end
print("集中度: ", tmp,"\n\n")

print("私立大學中出現次數最多的前十個名字: \n")
tmp = 0.0
for i in PsortDict[1:10]
    tmp += round(i[2]/P; digits = 4)
    print(i[1], " /出現次數: ", i[2], " /累積比例: ", round(tmp; digits = 4), "\n")
end
print("集中度: ", tmp)
print("\n\n私立學校集中度較高")

(f):
公立大學中出現次數最多的前十個名字: 
28219怡君 /出現次數: 73 /累積比例: 0.0026
雅婷 /出現次數: 72 /累積比例: 0.0052
宗翰 /出現次數: 59 /累積比例: 0.0073
家豪 /出現次數: 44 /累積比例: 0.0089
怡婷 /出現次數: 40 /累積比例: 0.0103
雅雯 /出現次數: 38 /累積比例: 0.0116
婉婷 /出現次數: 35 /累積比例: 0.0128
欣怡 /出現次數: 35 /累積比例: 0.014
佩珊 /出現次數: 35 /累積比例: 0.0152
佳蓉 /出現次數: 30 /累積比例: 0.0163
集中度: 0.0163

私立大學中出現次數最多的前十個名字: 
雅婷 /出現次數: 218 /累積比例: 0.0036
怡君 /出現次數: 183 /累積比例: 0.0066
欣怡 /出現次數: 101 /累積比例: 0.0083
怡婷 /出現次數: 100 /累積比例: 0.0099
佳蓉 /出現次數: 99 /累積比例: 0.0115
家豪 /出現次數: 94 /累積比例: 0.013
宗翰 /出現次數: 93 /累積比例: 0.0145
婉婷 /出現次數: 85 /累積比例: 0.0159
雅雯 /出現次數: 77 /累積比例: 0.0172
佩珊 /出現次數: 75 /累積比例: 0.0184
集中度: 0.0184

私立學校集中度較高